
# RAG with Chroma

1. 문서 읽기
2. 문서를 청크로 나눈다.
3. 문서를 임베딩한다. -> 벡터 데이터베이스에 저장
4. 질문이 있을때 벡터데이터 베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

## 1. 문서 읽기

In [12]:
%pip install -qU docx2txt langchain-community langchain-text-splitters


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 읽기
loader = Docx2txtLoader("./data/2023년도 맞춤형복지제도 업무처리 세부지침.docx")

# chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
document_list = loader.load_and_split(text_splitter)


/Users/hunchulchoi/.local/share/virtualenvs/rag를_활용한_llm_application개발-f5rS8os9/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
/Users/hunchulchoi/.local/share/virtualenvs/rag를_활용한_llm_application개발-f5rS8os9/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
 document_list


[Document(metadata={'source': './data/2023년도 맞춤형복지제도 업무처리 세부지침.docx'}, page_content='[붙임 \n\n\n\n\n\n\n\n[붙임 \n\n[붙임 \n\n\n\n\n\n\n\n\n\n2023년도 전라북도교육감 소속 공무원 \n\n맞춤형복지제도 업무처리 세부지침\n\n\n\n2023. 2.\n\n\n\n     행정국 재무과\n\nⅠ. 맞춤형 복지제도 개요·······················································1\n\nⅡ. 맞춤형복지 항목 설계·······················································2\n\nⅢ. 복지점수 배정 ·······························································6\n\n차  례\n\nⅣ. 복지점수 집행 및 정산···················································15\n\nⅤ. 행정사항·······································································16\n\n【붙임 1】 맞춤형복지비 적용 배제 또는 제한자 현황···························18\n\n【붙임 2】 2023년 온누리상품권 의무구매 비율 의견수렴 결과············19\n\n【붙임 3】 가족 복지점수 추가 배정 신청서···········································20\n\n【붙임 4】 출산축하 및 태아·산모검진 지원 복지점수 신청 서식················21\n\n【붙임 5】 난임지원 복지점수 신청 서식················································22\n\n\n\n2023년도 전라북도교육감 소속 공무원\n\n맞춤형복지제도 업무처리 세부지침\n\n전년 대비 주요 변동 내용\n\n

## embedding
### upstage

In [4]:
%pip install -qU langchain-upstage

  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [115 lines of output]
      Running `maturin pep517 build-wheel -i /Users/hunchulchoi/.local/share/virtualenvs/rag를_활용한_llm_application개발-f5rS8os9/bin/python --compatibility off`
      Rust not found, installing into a temporary directory
      Python reports SOABI: cpython-314-darwin
      Computed rustc target triple: aarch64-apple-darwin
      Installation directory: /Users/hunchulchoi/Library/Caches/puccinialin
      Rustup already downloaded
      Installing rust to /Users/hunchulchoi/Library/Caches/puccinialin/rustup
      warn: It looks like you have an existing rustup settings file at:
      warn: /Users/hunchulchoi/.rustup/settings.toml
      warn: Rustup will install the default toolchain as specified in the settings file,
      warn: instead of the one inferred from the default host triple.
      info: profile set to 'minimal

In [53]:
from sentence_transformers import SentenceTransformer

# embedding
embeddings = SentenceTransformer('jhgan/ko-sroberta-multitask')


In [ ]:
%pip install --upgrade langchain langchain-community langchain-chroma chromadb langchain-chroma

from langchain_chroma import Chroma

database = Chroma.from_documents(document_list, embeddings)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached langchain-1.1.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached langchain_chroma-1.0.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached chromadb-1.3.5-cp39-abi3-macosx_11_0_arm64.whl.metadata (7.2 kB)
  Using cached langgraph-1.0.4-py3-none-any.whl.metadata (7.8 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pybase64-1.4.2-cp314-cp314-macosx_11_0_arm64.whl.metadata (8.7 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
  Using cached chromadb-1.3.4-cp39-abi3-macosx_11_0_arm64.whl.metadata (7.2 kB)
  Using cached chromadb-1.3.3-cp39-abi3-macosx_11_0_arm64.whl.metadata (7.2 kB)
  Using cached chromadb-1.3.2-cp39-abi3-macosx_11_0_arm64.whl.metadata (7.2 kB)
  Using cached chromadb-1.3.0-cp39-abi3-macosx_11_0_arm64

In [57]:
query = '보험 필수 항목은 무엇인가요?'

retd_docs = database.similarity_search(query)

NameError: name 'database' is not defined